In [1]:
!pip install mediapipe opencv-python

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached jax-0.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.0-cp312-cp312-win_amd64.whl.metadata (1.4 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ml_dtypes-0.5.3-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.7.2-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.2-cp312-cp312-win_amd64.whl.metadata (1.4 kB)
  Using cached jax-0.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.1-cp312-cp312-win_amd64.whl.metadata (1.3 kB

In [5]:
import cv2
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

POSE_COLOR = (0, 255, 0)
LEFT_HAND_COLOR = (255, 0, 0)
RIGHT_HAND_COLOR = (0, 0, 255)

UPPER_COLOR = (255, 150, 0)    # ORANGE
LOWER_COLOR = (150, 0, 255)    # PURPLE

holistic = mp_holistic.Holistic(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)

cap = cv2.VideoCapture(0)

cv2.namedWindow("Tracker", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Tracker", 1920, 1080)  # almost fullscreen but with title bar

def body_too_close(landmarks):
    if not landmarks:
        return True
    visible_count = sum(1 for lm in landmarks.landmark if lm.visibility > 0.4)
    return visible_count < 15

UPPER_BODY_CONNECTIONS = [
    (11, 12),  # Shoulders
    (11, 13), (13, 15),  # Left arm
    (12, 14), (14, 16),  # Right arm
    (11, 23), (12, 24)   # Upper torso
]

LOWER_BODY_CONNECTIONS = [
    (23, 24),  # Hips
    (23, 25), (25, 27),  # Left leg
    (24, 26), (26, 28)   # Right leg
]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = holistic.process(rgb)

    if result.pose_landmarks:
        lm = result.pose_landmarks.landmark

        mp_drawing.draw_landmarks(
            frame,
            result.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=POSE_COLOR, thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=POSE_COLOR, thickness=2)
        )

        for a, b in UPPER_BODY_CONNECTIONS:
            ax, ay = int(lm[a].x * frame.shape[1]), int(lm[a].y * frame.shape[0])
            bx, by = int(lm[b].x * frame.shape[1]), int(lm[b].y * frame.shape[0])
            cv2.line(frame, (ax, ay), (bx, by), UPPER_COLOR, 4)

        for a, b in LOWER_BODY_CONNECTIONS:
            ax, ay = int(lm[a].x * frame.shape[1]), int(lm[a].y * frame.shape[0])
            bx, by = int(lm[b].x * frame.shape[1]), int(lm[b].y * frame.shape[0])
            cv2.line(frame, (ax, ay), (bx, by), LOWER_COLOR, 4)

    if result.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            result.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=LEFT_HAND_COLOR, thickness=2, circle_radius=3)
        )

    if result.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            result.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=RIGHT_HAND_COLOR, thickness=2, circle_radius=3)
        )

    if body_too_close(result.pose_landmarks):
        cv2.putText(frame, "MOVE AWAY FROM CAMERA",
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, (0, 0, 255), 4)

    cv2.imshow("Tracker", frame)

    if cv2.getWindowProperty("Tracker", cv2.WND_PROP_VISIBLE) < 1:
        break

    key = cv2.waitKey(1)
    if key & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()


# AI Fitness Trainer - Advanced Pose Detection

This notebook demonstrates:
- Real-time pose tracking with MediaPipe
- Automatic rep counting for squats and pushups
- Form scoring (0-100%)
- Live feedback on exercise form

In [ ]:
# Enhanced Pose Detection with Rep Counter and Form Analysis
import cv2
import mediapipe as mp
import numpy as np
import math

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Color scheme
POSE_COLOR = (0, 255, 0)
UPPER_COLOR = (255, 150, 0)
LOWER_COLOR = (150, 0, 255)

# Exercise state variables
exercise_mode = "squat"  # Options: squat, pushup
rep_count = 0
stage = None  # "up" or "down"
form_score = 0
feedback_messages = []

def calculate_angle(a, b, c):
    """Calculate angle between three points"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

def analyze_squat(landmarks):
    """Analyze squat form and count reps"""
    global rep_count, stage, form_score, feedback_messages
    
    # Get landmarks
    hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,
           landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
    knee = [landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].x,
            landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].y]
    ankle = [landmarks[mp_holistic.PoseLandmark.LEFT_ANKLE.value].x,
             landmarks[mp_holistic.PoseLandmark.LEFT_ANKLE.value].y]
    shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,
                landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
    
    # Calculate knee angle
    knee_angle = calculate_angle(hip, knee, ankle)
    
    # Calculate back angle (should be relatively straight)
    back_angle = calculate_angle(shoulder, hip, knee)
    
    feedback_messages = []
    score_components = []
    
    # Check depth
    if knee_angle < 90:
        if stage != "down":
            stage = "down"
        score_components.append(100)  # Good depth
    elif knee_angle < 120:
        score_components.append(70)
        feedback_messages.append("Go deeper")
    else:
        if stage == "down":
            stage = "up"
            rep_count += 1
        score_components.append(50)
    
    # Check back straightness (170-190 degrees is good)
    if 160 < back_angle < 200:
        score_components.append(100)
    else:
        score_components.append(50)
        feedback_messages.append("Keep back straight")
    
    # Check knee alignment
    knee_x = knee[0]
    ankle_x = ankle[0]
    if abs(knee_x - ankle_x) < 0.1:  # Knee aligned with ankle
        score_components.append(100)
    else:
        score_components.append(60)
        if knee_x > ankle_x:
            feedback_messages.append("Knees too forward")
    
    form_score = int(np.mean(score_components)) if score_components else 0
    
    return knee_angle, back_angle

def analyze_pushup(landmarks):
    """Analyze pushup form and count reps"""
    global rep_count, stage, form_score, feedback_messages
    
    # Get landmarks
    shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,
                landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,
             landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
    wrist = [landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x,
             landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
    hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,
           landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
    knee = [landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].x,
            landmarks[mp_holistic.PoseLandmark.LEFT_KNEE.value].y]
    
    # Calculate elbow angle
    elbow_angle = calculate_angle(shoulder, elbow, wrist)
    
    # Calculate body alignment (should be straight)
    body_angle = calculate_angle(shoulder, hip, knee)
    
    feedback_messages = []
    score_components = []
    
    # Check depth (elbow should go below 90 degrees)
    if elbow_angle < 90:
        if stage != "down":
            stage = "down"
        score_components.append(100)
    elif elbow_angle < 120:
        score_components.append(70)
        feedback_messages.append("Go lower")
    else:
        if stage == "down":
            stage = "up"
            rep_count += 1
        score_components.append(50)
    
    # Check body alignment (should be straight, around 170-190 degrees)
    if 160 < body_angle < 200:
        score_components.append(100)
    else:
        score_components.append(50)
        if body_angle < 160:
            feedback_messages.append("Hips too low")
        else:
            feedback_messages.append("Hips too high")
    
    form_score = int(np.mean(score_components)) if score_components else 0
    
    return elbow_angle, body_angle

def draw_stats(frame, rep_count, form_score, exercise_mode, feedback_messages):
    """Draw statistics overlay on frame"""
    # Semi-transparent background for stats
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (400, 200), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
    
    # Exercise mode
    cv2.putText(frame, f"Exercise: {exercise_mode.upper()}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Rep counter
    cv2.putText(frame, f"Reps: {rep_count}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)
    
    # Form score with color coding
    score_color = (0, 255, 0) if form_score > 80 else (0, 165, 255) if form_score > 60 else (0, 0, 255)
    cv2.putText(frame, f"Form: {form_score}%", (20, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 1, score_color, 2)
    
    # Feedback messages
    y_offset = 160
    for msg in feedback_messages:
        cv2.putText(frame, msg, (20, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
        y_offset += 30
    
    # Instructions
    cv2.putText(frame, "Press 'q': Squat | 'p': Pushup | 'r': Reset | 's': Stop",
                (10, frame.shape[0] - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

holistic = mp_holistic.Holistic(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)

cap = cv2.VideoCapture(0)
cv2.namedWindow("AI Fitness Trainer", cv2.WINDOW_NORMAL)
cv2.resizeWindow("AI Fitness Trainer", 1280, 720)

print("🏋️ AI Fitness Trainer Started!")
print("Controls:")
print("  'q' - Switch to Squat mode")
print("  'p' - Switch to Pushup mode")
print("  'r' - Reset rep counter")
print("  's' - Stop and exit")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = holistic.process(rgb)
    
    if result.pose_landmarks:
        lm = result.pose_landmarks.landmark
        
        # Draw pose skeleton
        mp_drawing.draw_landmarks(
            frame,
            result.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=POSE_COLOR, thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=POSE_COLOR, thickness=2)
        )
        
        # Analyze exercise based on mode
        if exercise_mode == "squat":
            analyze_squat(lm)
        elif exercise_mode == "pushup":
            analyze_pushup(lm)
    
    # Draw stats overlay
    draw_stats(frame, rep_count, form_score, exercise_mode, feedback_messages)
    
    cv2.imshow("AI Fitness Trainer", frame)
    
    if cv2.getWindowProperty("AI Fitness Trainer", cv2.WND_PROP_VISIBLE) < 1:
        break
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        break
    elif key == ord('q'):
        exercise_mode = "squat"
        rep_count = 0
        stage = None
        print("Switched to SQUAT mode")
    elif key == ord('p'):
        exercise_mode = "pushup"
        rep_count = 0
        stage = None
        print("Switched to PUSHUP mode")
    elif key == ord('r'):
        rep_count = 0
        stage = None
        print("Rep counter reset")

cap.release()
cv2.destroyAllWindows()

print(f"\n✅ Session Complete!")
print(f"   Total Reps: {rep_count}")
print(f"   Final Form Score: {form_score}%")